### OpenStreetMap Data Preprocessing - Extract Points of Interest Feature

In [0]:
#Mount google drive for content synchronization

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Install libraries for geospatial data wrangling and analysis

#Geopandas for wrangling
!pip install geopandas


#Contextily for basemap plotting 
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!pip3 install contextily==1.0rc2


     |████████████████████████████████| 931kB 2.8MB/s 
     |████████████████████████████████| 10.4MB 14.8MB/s 
     |████████████████████████████████| 14.7MB 51.2MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following NEW packages will be installed:
  libproj-dev proj-bin
0 upgraded, 2 newly installed, 0 to remove and 25 not upgraded.
Need to get 232 kB of archives.
After this operation, 1,220 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libproj-dev amd64 4.9.3-2 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 232 kB in 1s (375 kB/s)
Selecting previously unselected package libproj-dev:amd64.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../libproj-dev_4.9.3-2_amd64.deb ...


In [0]:
import geopandas
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame
import geopandas
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import contextily

#pois_df = geopandas.read_file('/content/drive/My Drive/Thesis/Bangladesh OSM Data/gis_osm_pois_free_1.shp')
#pois_df.tail()

In [0]:
import pickle
import os
os.chdir('/content/drive/My Drive/Thesis/OSM_data')

with open('clusters_rural_pois', 'rb') as fp:
  clusters_rural_pois = pickle.load(fp)

with open('clusters_urban_pois', 'rb') as fp:
  clusters_urban_pois = pickle.load(fp)


In [0]:
#Load cluster data
import pandas as pd
clusters_df = pd.read_csv('/content/drive/My Drive/Thesis/DHS_Data/bangladesh_cluster_avg_asset_2013_updated_new.csv')
clusters_df = geopandas.GeoDataFrame(clusters_df, geometry = geopandas.points_from_xy(clusters_df.longitude, clusters_df.latitude))
clusters_df.tail()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
594,595,596,1.757480,U,24.900228,91.871489,POINT (91.87149 24.90023)
595,596,597,1.591590,U,24.904567,91.887165,POINT (91.88716 24.90457)
596,597,598,2.281590,U,24.886183,91.887103,POINT (91.88710 24.88618)
597,598,599,-0.042325,U,24.893073,91.906950,POINT (91.90695 24.89307)
598,599,600,1.242605,U,24.875934,91.895091,POINT (91.89509 24.87593)


*italicized text*### OSM Point of Interest Data Feature Engineering 

In [0]:
#Separate Urban and Rural area clusters

clusters_rural = clusters_df[clusters_df['URBAN_RURA'] == 'R']
clusters_urban = clusters_df[clusters_df['URBAN_RURA'] == 'U']
clusters_rural.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
0,0,1,-0.831240,R,21.965697,90.126312,POINT (90.12631 21.96570)
1,1,2,-0.739180,R,22.221232,90.348908,POINT (90.34891 22.22123)
2,2,3,-0.786210,R,22.191484,90.212539,POINT (90.21254 22.19148)
3,3,4,-0.447090,R,22.185481,90.167023,POINT (90.16702 22.18548)
4,4,5,-0.687435,R,22.311173,90.149822,POINT (90.14982 22.31117)


In [0]:
clusters_urban.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry
50,50,51,0.145555,U,22.302611,90.091872,POINT (90.09187 22.30261)
51,51,52,0.913850,U,22.053795,89.968962,POINT (89.96896 22.05380)
52,52,53,-0.261730,U,22.813022,90.191408,POINT (90.19141 22.81302)
53,53,54,-0.231740,U,22.691392,90.389866,POINT (90.38987 22.69139)
54,54,55,1.585980,U,22.691265,90.640443,POINT (90.64044 22.69127)


In [0]:
type(clusters_rural.geometry[0])

shapely.geometry.point.Point

In [0]:
#convert Rural clusters to buffers with 5km radius
clusters_rural.crs = {'init' :'epsg:4326'}
clusters_rural= clusters_rural.to_crs(epsg=3174)  
buffer_length_in_meters = (5 * 1000)

rural_buffer = clusters_rural.geometry.buffer(buffer_length_in_meters)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
len(rural_buffer)

392

In [0]:
#convert Urban clusters to buffers with 2km radius
clusters_urban.crs = {'init' :'epsg:4326'}
clusters_urban= clusters_urban.to_crs(epsg=3174)  
buffer_length = (2 * 1000)

urban_buffer = clusters_urban.geometry.buffer(buffer_length)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [0]:
len(urban_buffer)

207

In [0]:
#Convert roads_df to same crs as Urban and Rural clusters df
pois_df = pois_df.to_crs(epsg=3174)

In [0]:
#Helper function to calculate the number of roads within a buffer zone of a cluster

def pois_within(buffer,pois_df): 
  no_pois_list = []
  for point in buffer:
    no_pois = pois_df.within(point)
    no_pois_list.append(no_pois.sum())
  return no_pois_list

In [0]:
#Create list of different POIS types

pois_type_name = pois_df.fclass.unique()
pois_type_name

array(['pub', 'hospital', 'school', 'cinema', 'zoo', 'monument',
       'nightclub', 'playground', 'police', 'college', 'hotel',
       'restaurant', 'supermarket', 'town_hall', 'atm', 'university',
       'embassy', 'bench', 'viewpoint', 'attraction', 'bank', 'fast_food',
       'tower', 'camp_site', 'lighthouse', 'post_office', 'mall',
       'comms_tower', 'hostel', 'fire_station', 'guesthouse',
       'theme_park', 'public_building', 'beverages', 'post_box', 'castle',
       'pharmacy', 'car_rental', 'convenience', 'theatre', 'cafe',
       'library', 'sports_centre', 'car_sharing', 'memorial', 'bakery',
       'drinking_water', 'waste_basket', 'telephone', 'florist',
       'laundry', 'gift_shop', 'car_dealership', 'bookshop', 'clothes',
       'jeweller', 'butcher', 'hairdresser', 'park', 'shoe_shop', 'pitch',
       'prison', 'toilet', 'swimming_pool', 'picnic_site', 'stadium',
       'courthouse', 'community_centre', 'motel', 'fountain', 'museum',
       'graveyard', 'car_wash'

In [0]:
#Create list of dataframes of different POIS types
pois_type = []
for i in range(len(pois_type_name)): 
  pois_type_list = pois_df[pois_df['fclass']== pois_type_name[i]]
  pois_type.append(pois_type_list)

In [0]:
##Engineer the total number of roads feature for each road types and append to rural dataframe

for i in range(len(pois_type)):
  pois_rural = pois_within(rural_buffer, pois_type[i])
  pois_rural = np.asarray(pois_rural)
  clusters_rural[f"Count of {pois_type_name[i]}"] = pois_rural

In [0]:
clusters_rural.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry,Count of pub,Count of hospital,Count of school,Count of cinema,Count of zoo,Count of monument,Count of nightclub,Count of playground,Count of police,Count of college,Count of hotel,Count of restaurant,Count of supermarket,Count of town_hall,Count of atm,Count of university,Count of embassy,Count of bench,Count of viewpoint,Count of attraction,Count of bank,Count of fast_food,Count of tower,Count of camp_site,Count of lighthouse,Count of post_office,Count of mall,Count of comms_tower,Count of hostel,Count of fire_station,Count of guesthouse,Count of theme_park,Count of public_building,...,Count of travel_agent,Count of arts_centre,Count of newsagent,Count of veterinary,Count of kindergarten,Count of dentist,Count of shelter,Count of beauty_shop,Count of water_works,Count of wastewater_plant,Count of recycling,Count of food_court,Count of bar,Count of tourist_info,Count of stationery,Count of track,Count of chemist,Count of video_shop,Count of observation_tower,Count of greengrocer,Count of toy_shop,Count of camera_surveillance,Count of vending_any,Count of artwork,Count of hunting_stand,Count of sports_shop,Count of water_mill,Count of nursing_home,Count of optician,Count of garden_centre,Count of archaeological,Count of alpine_hut,Count of outdoor_shop,Count of golf_course,Count of wayside_shrine,Count of windmill,Count of vending_machine,Count of dog_park,Count of bicycle_rental,Count of wayside_cross
0,0,1,-0.831240,R,21.965697,90.126312,POINT (8280551.707 12256784.033),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,-0.739180,R,22.221232,90.348908,POINT (8244779.973 12261806.575),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,3,-0.786210,R,22.191484,90.212539,POINT (8255805.397 12251265.056),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,-0.447090,R,22.185481,90.167023,POINT (8259144.580 12247509.773),0,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,5,-0.687435,R,22.311173,90.149822,POINT (8249404.697 12238539.969),0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#Dump file for later integration with original dataframe
import os
import pickle

os.chdir('/content/drive/My Drive/Thesis/OSM_data')

with open('clusters_rural_pois', 'wb') as fp:
    pickle.dump(clusters_rural, fp)

In [0]:
##Engineer the total number of roads feature for each road types and append to rural dataframe

for i in range(len(pois_type)):
  pois_urban = pois_within(urban_buffer, pois_type[i])
  pois_urban = np.asarray(pois_urban)
  clusters_urban[f"Count of {pois_type_name[i]}"] = pois_urban

In [0]:
#Dump file for later integration with original dataframe
import pickle

with open('clusters_urban_pois', 'wb') as fp:
    pickle.dump(clusters_urban, fp)

In [0]:
import pickle
with open('clusters_rural_pois', 'rb') as fp:
  clusters_rural = pickle.load(fp)

In [0]:
clusters_rural.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry,Count of pub,Count of hospital,Count of school,Count of cinema,Count of zoo,Count of monument,Count of nightclub,Count of playground,Count of police,Count of college,Count of hotel,Count of restaurant,Count of supermarket,Count of town_hall,Count of atm,Count of university,Count of embassy,Count of bench,Count of viewpoint,Count of attraction,Count of bank,Count of fast_food,Count of tower,Count of camp_site,Count of lighthouse,Count of post_office,Count of mall,Count of comms_tower,Count of hostel,Count of fire_station,Count of guesthouse,Count of theme_park,Count of public_building,...,Count of travel_agent,Count of arts_centre,Count of newsagent,Count of veterinary,Count of kindergarten,Count of dentist,Count of shelter,Count of beauty_shop,Count of water_works,Count of wastewater_plant,Count of recycling,Count of food_court,Count of bar,Count of tourist_info,Count of stationery,Count of track,Count of chemist,Count of video_shop,Count of observation_tower,Count of greengrocer,Count of toy_shop,Count of camera_surveillance,Count of vending_any,Count of artwork,Count of hunting_stand,Count of sports_shop,Count of water_mill,Count of nursing_home,Count of optician,Count of garden_centre,Count of archaeological,Count of alpine_hut,Count of outdoor_shop,Count of golf_course,Count of wayside_shrine,Count of windmill,Count of vending_machine,Count of dog_park,Count of bicycle_rental,Count of wayside_cross
0,0,1,-0.831240,R,21.965697,90.126312,POINT (8280551.707 12256784.033),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,-0.739180,R,22.221232,90.348908,POINT (8244779.973 12261806.575),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,3,-0.786210,R,22.191484,90.212539,POINT (8255805.397 12251265.056),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,-0.447090,R,22.185481,90.167023,POINT (8259144.580 12247509.773),0,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,5,-0.687435,R,22.311173,90.149822,POINT (8249404.697 12238539.969),0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import pickle
with open('clusters_urban_pois', 'rb') as fp:
  clusters_urban = pickle.load(fp)

In [0]:
clusters_urban.head()

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry,Count of pub,Count of hospital,Count of school,Count of cinema,Count of zoo,Count of monument,Count of nightclub,Count of playground,Count of police,Count of college,Count of hotel,Count of restaurant,Count of supermarket,Count of town_hall,Count of atm,Count of university,Count of embassy,Count of bench,Count of viewpoint,Count of attraction,Count of bank,Count of fast_food,Count of tower,Count of camp_site,Count of lighthouse,Count of post_office,Count of mall,Count of comms_tower,Count of hostel,Count of fire_station,Count of guesthouse,Count of theme_park,Count of public_building,...,Count of travel_agent,Count of arts_centre,Count of newsagent,Count of veterinary,Count of kindergarten,Count of dentist,Count of shelter,Count of beauty_shop,Count of water_works,Count of wastewater_plant,Count of recycling,Count of food_court,Count of bar,Count of tourist_info,Count of stationery,Count of track,Count of chemist,Count of video_shop,Count of observation_tower,Count of greengrocer,Count of toy_shop,Count of camera_surveillance,Count of vending_any,Count of artwork,Count of hunting_stand,Count of sports_shop,Count of water_mill,Count of nursing_home,Count of optician,Count of garden_centre,Count of archaeological,Count of alpine_hut,Count of outdoor_shop,Count of golf_course,Count of wayside_shrine,Count of windmill,Count of vending_machine,Count of dog_park,Count of bicycle_rental,Count of wayside_cross
50,50,51,0.145555,U,22.302611,90.091872,POINT (8253726.377 12233816.998),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51,51,52,0.913850,U,22.053795,89.968962,POINT (8282735.800 12237347.209),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52,52,53,-0.261730,U,22.813022,90.191408,POINT (8203633.163 12212611.699),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
53,53,54,-0.231740,U,22.691392,90.389866,POINT (8201852.534 12237607.774),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
54,54,55,1.585980,U,22.691265,90.640443,POINT (8186321.635 12260042.307),0,0,1,0,0,1,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Merge engineered features with the original POIS data

In [0]:
merged = pd.concat([clusters_rural_pois, clusters_urban_pois])
merged = merged.sort_values(by = 'cluster')

,Unnamed: 0,cluster,wlthindf,URBAN_RURA,latitude,longitude,geometry,Count of pub,Count of hospital,Count of school,Count of cinema,Count of zoo,Count of monument,Count of nightclub,Count of playground,Count of police,Count of college,Count of hotel,Count of restaurant,Count of supermarket,Count of town_hall,Count of atm,Count of university,Count of embassy,Count of bench,Count of viewpoint,Count of attraction,Count of bank,Count of fast_food,Count of tower,Count of camp_site,Count of lighthouse,Count of post_office,Count of mall,Count of comms_tower,Count of hostel,Count of fire_station,Count of guesthouse,Count of theme_park,Count of public_building,...,Count of travel_agent,Count of arts_centre,Count of newsagent,Count of veterinary,Count of kindergarten,Count of dentist,Count of shelter,Count of beauty_shop,Count of water_works,Count of wastewater_plant,Count of recycling,Count of food_court,Count of bar,Count of tourist_info,Count of stationery,Count of track,Count of chemist,Count of video_shop,Count of observation_tower,Count of greengrocer,Count of toy_shop,Count of camera_surveillance,Count of vending_any,Count of artwork,Count of hunting_stand,Count of sports_shop,Count of water_mill,Count of nursing_home,Count of optician,Count of garden_centre,Count of archaeological,Count of alpine_hut,Count of outdoor_shop,Count of golf_course,Count of wayside_shrine,Count of windmill,Count of vending_machine,Count of dog_park,Count of bicycle_rental,Count of wayside_cross
0,0,1,-0.831240,R,21.965697,90.126312,POINT (8280551.707 12256784.033),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,-0.739180,R,22.221232,90.348908,POINT (8244779.973 12261806.575),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,3,-0.786210,R,22.191484,90.212539,POINT (8255805.397 12251265.056),0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,-0.447090,R,22.185481,90.167023,POINT (8259144.580 12247509.773),0,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,5,-0.687435,R,22.311173,90.149822,POINT (8249404.697 12238539.969),0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,595,596,1.757480,U,24.900228,91.871489,POINT (7920498.952 12234150.119),0,7,9,0,0,3,0,0,1,6,9,34,3,0,21,0,0,0,0,2,37,5,1,0,0,1,9,0,7,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
595,596,597,1.591590,U,24.904567,91.887165,POINT (7919157.386 12235232.377),0,3,6,0,0,2,0,1,0,2,3,12,1,0,7,2,0,0,0,0,17,5,0,0,0,0,2,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
596,597,598,2.281590,U,24.886183,91.887103,POINT (7920737.339 12236359.585),0,2,3,0,0,1,0,1,0,2,2,11,2,0,8,0,0,0,0,0,17,5,0,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
597,598,599,-0.042325,U,24.893073,91.906950,POINT (7918918.634 12237643.678),0,1,4,0,0,0,0,1,1,1,0,4,0,0,3,1,0,0,0,0,9,2,0,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
merged.isnull().values.any()

False

In [0]:
merged.to_csv('clusters_pois.csv')